[JSON-schema](json-schema.ipynb) <- vorige - [Inhoudsopgave](Inhoud.ipynb) - volgende -> [Linked data(2)](Linked-data-2.ipynb)

# Linked data en JSON-LD

## Inleiding

In de voorgaande hoofdstukken heb je gezien hoe je met een schema ervoor kunt zorgen dat de documenten in een MongoDB database-collection gelijksoortig zijn.
In MongoDB kun je een query als schema koppelen aan een collection, om een document te valideren voordat je dit toevoegt of verandert.

In plaats van een MongoDB-query kun je ook de JSON-schema notatie gebruiken.
Hiermee kun je nog preciezer beschrijven wat de structuur van een document moet zijn, welke (veld)namen gebruikt moeten worden, en hoe de waarden getypeerd moeten zijn.

Door het gebruik van standaard-schema's zoals die van http://schema.org kun je ervoor zorgen dat de betekenis van de velden duidelijk vastligt, in elk geval voor de menselijke lezer. 
En je zorgt ervoor dat je documenten uitwisselbaar zijn met anderen die dezelfde schema's gebruiken.

Een volgende vraag is hoe je de eigenschappen van een schema expliciet kunt maken, zodat deze ook gebruikt (geïnterpreteerd) kunnen worden door computers, en niet alleen door mensen. Hiervoor kun je gebruik maken van de Linked Data technologie zoals die ontwikkeld is voor het web.

* hoe kun je duidelijk maken welke standaard-schema's je gebruikt?
* hoe kun je het verband met andere data aangeven?


## Linked data

Volgens Tim Berners-Lee, de uitvinder van het web, bestaat Linked Data uit de volgende stappen:
    
zie: https://www.w3.org/DesignIssues/LinkedData.html

1. Use URIs as names for things
2. Use HTTP URIs so that people can look up those names.
3. When someone looks up a URI, provide useful information, using the standards (RDF*, SPARQL)
4. Include links to other URIs. so that they can discover more things.    

## (1) Gebruik URI's/IRI's als namen

De eerste stap is het gebruiken van globale unieke identificaties voor "dingen" (resources).
Dergelijke globale identificaties zijn iet afhankelijk van de lokale context:
Dit betekent dat je gegevens gemakkelijk kunt uitwisselen tussen verschillende lokale contexten.
In het web is voor deze globale identificatie de URI (tegenwoordig: IRI) ontwikkeld.

> De web-term voor "ding" is "resource": dit kan iets concreets zijn: een ding, mens, dier, planeet; het kan ook iets virtueels of abstracts zijn. Het hoeft niet in het internet/web aanwezig te zijn, om het in het web te kunnen beschrijven.

Enkele voorbeelden van URI's/IRI's:

* urn:isbn:0-486-27557-4  (een ISBN-nummer identificeert een boek-editie)
* urn:dev:mac:0024befffe804ff1 (een MAC-adres identificeert een apparaat/netwerk-aansluiting)
* tel:+1-816-555-1212
* mailto:John.Doe@example.com
* ftp://ftp.is.co.za/rfc/rfc1808.txt
* http://www.ietf.org/rfc/rfc2396.txt

De eerste van deze voorbeelden zijn identificaties die je niet als link in het web kunt gebruiken.
Deze vallen onder de categorie "Uniform Resource Numbers" (urn).
De laatste voorbeelden, vanaf het telefoonnummer, kun je wel als link in een webpagina gebruiken.
De browser kan deze identificaties interpreteren, en er actie op ondernemen.
Bij de laatste twee gevallen kun je via de browser meer gegevens bij de identificatie ophalen.

> Het belangrijkste verschil tussen URI's en IRI's is dat deze laatste veel meer mogelijkheden bieden om een alternatief alfabet te gebruiken. URI's zijn traditioneel sterk gericht op het latijnse alfabet (ASCII).

* https://tools.ietf.org/html/rfc3986 (internet-RFC over URI's)
* https://tools.ietf.org/html/rfc3987 (IRI's)
* https://en.wikipedia.org/wiki/Internationalized_Resource_Identifier

## (2) Gebruik URL's als URI's

De tweede stap is: gebruik waar mogelijk URL's als URI's.
Een URL kun je als link in het web gebruiken, en via een URL kun je meer gegevens bij de identificatie vinden (zie de volgende stap).

Je kunt dit toepassen op de veldnamen van een document.
Als je de namen gebruiken van schema.org krijgen je bijvoorbeeld:

```
{
    "http://schema.org/name": "Harry van Doorn",
    "http://schema.org/email": "harryvdoorm@friendmail.org",
    "http://schema.org/telephone": "+31-6-1357 8642"
}
```

Dit is voor computers duidelijk en ondubbelzinnig, maar voor mensen wat omslachtig.
Met behulp van JSON-LD kun je dit compacter en overzichtelijker beschrijven.

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from pyld import jsonld

## JSON-LD

In JSON-LD kun je bij een JSON-object aangeven welke context gebruikt wordt.
Anders gezegd: waar je de definitie van de gebruikte namen kunt vinden.
Het bovenstaande voorbeeld wordt dan:

```
{
  "@context: "http://schema.org",
  "name": "Harry van Doorn",
  "email": "harryvdoorm@friendmail.org",
  "telephone": "+31-6-1357 8642"
}
```

Met behulp van de hulpprogramma's bij JSON-LD (uit pyld) kun je dit expanderen.

> Door een fout in de python-implementatie moet je hier in plaats van "http://schmema.org" gebruiken: "http://schema.org/docs/jsonldcontext.jsonld". In een volgende versie is dat waarschijnlijk opgelost.

In [ ]:
mycontact = {
  "@context": "http://schema.org/docs/jsonldcontext.jsonld",
  "name": "Harry van Doorn",
  "email": "harryvdoorm@friendmail.org",
  "telephone": "+31-6-1357 8642"
}

expanded_contact = jsonld.expand(mycontact)
expanded_contact

Het omgekeerde is ook mogelijk: je kunt uit een geëxpandeerd document en een context weer een compact document maken.

> De volgorde van de velden in het document kan hierbij veranderen, maar dat heeft geen betekenis, noch in JSON, noch in Python.

In [ ]:
jsonld.compact(expanded_contact, "http://schema.org/docs/jsonldcontext.jsonld")

**Opdrachten**

* Ga na wat er gebeurt als je een naam gebruikt die niet in schema.org gedefinieerd wordt, bijvoorbeeld "tel" in plaats van "telephone".
    * welke URL krijg je als veldnaam?
    * wat gebeurt er als je die URL gebruikt in de browser? (als je op die URL klikt?, anders gezegd: als je het document bij die URL ophaalt?)
* Vraag de lijst met namen in de browser op, via de link http://schema.org/docs/jsonldcontext.jsonld

### Links

* https://json-ld.org
* https://json-ld.org/playground/

## (3) Geef extra informatie via de URL's

De volgende stap is om via de URL's die als naam gebruikt worden, meer informatie over de naam en het bijbehorende "ding" te geven.

Welke gegevens krijg je bij de URL "[http://schema.org/email](http://schema.org/email)"?

> We gebruiken hier de Python-library `requests`, voor het versturen van HTTP-requests vanuit Python. (zie: https://requests.readthedocs.io/)

In [ ]:
r = requests.get("http://schema.org/email")
r.status_code

Als de status_code = 200 ("OK"), dan was de aanvraag succesvol; de webserver heeft een document teruggestuurd, je vindt dat onder `r.text`. We laten hier de eerste 500 tekens zien:

In [ ]:
r.text[0:500]

Dit is hetzelfde HTML-document dat je krijgt als je die link in de browser opent.
Voor computers is dit document niet erg handig, maar er zit een JSON-LD document "verstopt" in de HTML-code.

Via de Python-library BeautifulSoup (REF), voor het verwerken van HTML-documenten, kun je die gegevens vinden, als een script met als type: "application/ld+json"):

In [ ]:
soup =  BeautifulSoup(r.text, "html.parser")
json_script = soup.find(type="application/ld+json")
json_script

Het JSON-LD object in dit script krijg je via:

In [ ]:
json.loads(json_script.contents[0].string)

**Opdracht**

* vergelijk het JSON-LD object met de gegevens die je via de browser vindt op [http://schema.org/email](http://schema.org/email).
* wat is het type van de waarde van het veld [http://schema.org/email](http://schema.org/email)?
* in welk soort documenten kan het veld [http://schema.org/email](http://schema.org/email) voorkomen?
* wat is het "label" bij [http://schema.org/email](http://schema.org/email)?
* wat is de betekenis van "label" volgens het bijbehorende schema?

### Functie get_json_ld

We definiëren een functie om een dergelijk JSON-LD script uit het HTML-bestand bij een gegeven URL te vinden, als dit er is. Het resultaat None geeft aan dat we bij die URL geen JSON-LD object kunnen vinden.

In [ ]:
def get_json_ld (url):
    r = requests.get(url)
    if r.status_code != 200: 
        return None
    soup =  BeautifulSoup(r.text, "html.parser")
    json_script = soup.find(type="application/ld+json")
    if json_script == None:
        return None
    return json.loads(json_script.contents[0].string)

**Opdrachten**

* zoek de gegevens op van [http://schema.org/productionDate](http://schema.org/productionDate)
    * via de browser
    * via de functie `get_json_ld`
* zoek de gegevens op van [http://schema.org/Person](http://schema.org/Person)
    * via de browser
    * via de functie `get_json_ld`

## (4) Gebruik links in de gegevens bij een URL

In het web gebruik je links om de verschillende documenten aan elkaar te koppelen.
In het geval van Linked Data kun je dat ook in de data zelf doen.
Dit heb je al gezien voor de gegevens in de schema's.
Je kunt het ook voor de waarden zelf doen.

We geven een aantal voorbeelden van het gebruik van Linked Data.
Ga zelf na hoe daarin links naar andere data gebruikt worden.

## JSON-LD voor zoekmachines

Met behulp van gestructureerde gegevens kunnen zoekmachines veel preciezer zoeken,
en zoekresultaten beter representeren.

Google beschrijft in de [Google structured data richtlijnen](https://developers.google.com/search/docs/guides/intro-structured-data) hoe je deze json-lddata in je webpagina's kunt opnemen.

We geven hieronder enkele voorbeelden van websites met ingebedde json-ld gegevens

In [ ]:
get_json_ld("https://ieni.org")

In [ ]:
get_json_ld("https://www.smulweb.nl/recepten/941502/Gekookt-ei")

In [ ]:
get_json_ld("https://nl.wikipedia.org/wiki/Hedy_Lamarr")

**Opdrachten**

* zoek een website van een winkel in de buurt waarin json-ld data gebruikt wordt. Welke gegevens vind je in die data?
* zoek via een zoekmachine naar die winkel; welke gegevens krijg je als resultaat van de zoekopdracht te zien?

## Het web van linked data

Zoals het web een web van documenten vormt, geeft linked data een web van data.
De [Linked Open Data Cloud](https://lod-cloud.net) is daarvan een deel met vrij toegankelijke en herbruikbare ("open") data.

* Wikipedia bevat naast de tekstuele data, ook gestructureerde data 
* [Wikidata](https://wikidata.org) is de open data-wiki van Mediawiki (Wikipedia)
* [DBpedia](https://dbpedia.org) is een open data-wiki op basis van de gegevens in Wikipedia en andere bronnen.

Je kunt deze gegevens op verschillende manieren benaderen, bijvoorbeeld via SPARQL queries.

*Opmerking*: deze gegevens zijn bedoeld om door computers gelezen te worden; voor mensen zijn de resultaten niet altijd handig.

Deze linked data vormt de eerste stap naar het Semantische web.

**Opdrachten**

* zoek Hedy Lamarr op Wikidata
* zoek Hedy Lamarr op DBPedia
    * gebruik bijvoorbeeld: http://dbpedia.org/fct/facet.vsp
    * wat is haar rol in "How we got to now"?

-----